In [2]:
import json
from tqdm import tqdm

import pandas
import networkx as nx
from pyvis.network import Network

from operator import itemgetter
from datetime import datetime

In [3]:
filepath = r'C:\Users\A101889\Documents\GitHub\SecurityAnalytics\Playground\sample_dataset\mimikatz_logonpasswords_2020-08-07103224.json'

with open(filepath) as f:
    readfile = f.readlines()
    
sampledata = json.loads(readfile[0]) 
sampledata

{'tags': ['mordorDataset'],
 'TargetProcessGUID': '{9f85ce58-5a6a-5f2b-a900-000000000400}',
 '@version': '1',
 'EventType': 'INFO',
 'Version': 3,
 'ThreadID': 4208,
 'EventTime': '2020-08-07 10:32:22',
 'Task': 10,
 'AccountType': 'User',
 'SourceProcessGUID': '{9f85ce58-59dc-5f2b-1000-000000000400}',
 'CallTrace': 'C:\\windows\\SYSTEM32\\ntdll.dll+9fc24|C:\\windows\\System32\\KERNELBASE.dll+6c175|c:\\windows\\system32\\lsm.dll+e575|C:\\windows\\System32\\RPCRT4.dll+77803|C:\\windows\\System32\\RPCRT4.dll+db4a6|C:\\windows\\System32\\RPCRT4.dll+10d20|C:\\windows\\System32\\RPCRT4.dll+54a18|C:\\windows\\System32\\RPCRT4.dll+30490|C:\\windows\\System32\\RPCRT4.dll+2fe3b|C:\\windows\\System32\\RPCRT4.dll+221ff|C:\\windows\\System32\\RPCRT4.dll+2165a|C:\\windows\\System32\\RPCRT4.dll+20c21|C:\\windows\\System32\\RPCRT4.dll+20692|C:\\windows\\System32\\RPCRT4.dll+17465|C:\\windows\\SYSTEM32\\ntdll.dll+4f4d0|C:\\windows\\SYSTEM32\\ntdll.dll+50358|C:\\windows\\System32\\KERNEL32.DLL+17974|C:

In [5]:
uniqset = set()
for data in readfile:
    jsonobj = json.loads(data) 
    for keys in jsonobj.keys():
        tempp = keys.lower()
        matches = ["message"]
        if any(x in tempp for x in matches):
            uniqset.add(keys)

uniqset = list(uniqset)
sorted(uniqset)

['Message']

In [70]:
#find all unique hosts
uniqueHosts = set()
atrName = "Hostname"
for item in tqdm(readfile):
    jsonobj = json.loads(item)
    if atrName in jsonobj.keys():
        uniqueHosts.add(str(jsonobj[atrName]))

list(uniqueHosts)

100%|███████████████| 6026/6026 [00:00<00:00, 45762.20it/s]


['WORKSTATION5.theshire.local',
 'WORKSTATION6.theshire.local',
 'MORDORDC.theshire.local']

In [69]:
asd = []
for item in tqdm(readfile):
    jsonobj = json.loads(item)
    if 'WORKSTATION5.theshire.local' in jsonobj.values():
        asd.append(jsonobj)
asd[0]

100%|███████████████| 6026/6026 [00:00<00:00, 31984.36it/s]


{'tags': ['mordorDataset'],
 '@version': '1',
 'EventType': 'CreateKey',
 'Version': 2,
 'ThreadID': 4364,
 'EventTime': '2020-08-07 10:32:22',
 'Task': 12,
 'AccountType': 'User',
 'Channel': 'Microsoft-Windows-Sysmon/Operational',
 'EventTypeOrignal': 'INFO',
 'SourceName': 'Microsoft-Windows-Sysmon',
 'Hostname': 'WORKSTATION5.theshire.local',
 '@timestamp': '2020-08-07T14:32:25.359Z',
 'OpcodeValue': 0,
 'Message': 'Registry object added or deleted:\r\nRuleName: -\r\nEventType: CreateKey\r\nUtcTime: 2020-08-07 14:32:22.822\r\nProcessGuid: {297bc33e-65d0-5f2d-8207-000000000400}\r\nProcessId: 1648\r\nImage: C:\\windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe\r\nTargetObject: HKU\\S-1-5-21-1363495622-3806888128-621328882-1104\\Software\\Microsoft\\Windows\\CurrentVersion\\Internet Settings\\Connections',
 'SourceModuleName': 'eventlog',
 'ProcessId': '1648',
 'EventReceivedTime': '2020-08-07 10:32:25',
 'port': 60122,
 'AccountName': 'SYSTEM',
 'UtcTime': '2020-08-07 14:32:

In [73]:
#find all assosiciation
edgeset = []

for data in tqdm(readfile):
    jsonobj = json.loads(data)
    for i in uniqueHosts:
        for j in uniqset:
            if i in jsonobj.values() and j in jsonobj.keys():
                edgeset.append((str(i),  str(jsonobj[j]).split("\\")[-1]) )
len(edgeset)

100%|███████████████| 6026/6026 [00:00<00:00, 11356.50it/s]


22379

In [74]:
edgeset

[('MORDORDC.theshire.local', 'svchost.exe'),
 ('MORDORDC.theshire.local', 'eventlog'),
 ('MORDORDC.theshire.local', '3052'),
 ('MORDORDC.theshire.local', 'im_msvistalog'),
 ('MORDORDC.theshire.local', 'Microsoft-Windows-Sysmon'),
 ('MORDORDC.theshire.local', 'svchost.exe'),
 ('MORDORDC.theshire.local', '{9f85ce58-59dc-5f2b-1000-000000000400}'),
 ('MORDORDC.theshire.local', '992'),
 ('MORDORDC.theshire.local', '{9f85ce58-5a6a-5f2b-a900-000000000400}'),
 ('MORDORDC.theshire.local', '340'),
 ('MORDORDC.theshire.local', 'svchost.exe'),
 ('MORDORDC.theshire.local', 'eventlog'),
 ('MORDORDC.theshire.local', '3052'),
 ('MORDORDC.theshire.local', 'im_msvistalog'),
 ('MORDORDC.theshire.local', 'Microsoft-Windows-Sysmon'),
 ('MORDORDC.theshire.local', 'svchost.exe'),
 ('MORDORDC.theshire.local', '{9f85ce58-59dc-5f2b-1000-000000000400}'),
 ('MORDORDC.theshire.local', '992'),
 ('MORDORDC.theshire.local', '{9f85ce58-5a6a-5f2b-a900-000000000400}'),
 ('MORDORDC.theshire.local', '340'),
 ('MORDORDC.th

In [75]:
# create the graph from edge list
G = nx.DiGraph()
G.add_edges_from(edgeset)
G.remove_edges_from(nx.selfloop_edges(G))

In [76]:
len(G)

636

In [77]:
# view the result in pyviz
nt = Network()
nt.show_buttons()
nt.from_nx(G)
nt.show("nx.html")

In [84]:
selected_data = dict( (n,d['EventID']) for n,d in G.nodes().items() if d['EventID'] == '12')

KeyError: 'EventID'

In [78]:
# Network density - corelation probability
density = nx.density(G)
print("Network density:", density)

Network density: 0.0016589907393651266


In [80]:
# shortest path - reachability
shortest_path = nx.shortest_path(G, source="1648", target="1824")

print("Shortest path from source to target is:", shortest_path)

NetworkXNoPath: No path between 1648 and 1824.

In [81]:
# clustering coefficient - triadic closure/ transitivity
triadic_closure = nx.transitivity(G)
print("Triadic closure:", triadic_closure)

Triadic closure: 0


In [83]:
# centrality measures - Find the nodes that is interating with other nodes in abnormal way
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by degree:")
for d in sorted_degree[:20]:
    print(d)

Top 20 nodes by degree:
('svchost.exe', 0.004724409448818898)
('eventlog', 0.004724409448818898)
('im_msvistalog', 0.004724409448818898)
('Microsoft-Windows-Sysmon', 0.004724409448818898)
('Connections', 0.004724409448818898)
('0.0.0.0', 0.004724409448818898)
('Microsoft-Windows-Security-Auditing', 0.004724409448818898)
('-', 0.004724409448818898)
('Parameters', 0.004724409448818898)
('lastalive0.dat', 0.004724409448818898)
('0x4', 0.004724409448818898)
('lastalive1.dat', 0.004724409448818898)
('172.18.39.5', 0.0031496062992125984)
('false', 0.0031496062992125984)
('::', 0.0031496062992125984)
('0x0', 0.0031496062992125984)
('S-1-5-18', 0.0031496062992125984)
('THESHIRE', 0.0031496062992125984)
('56142', 0.0031496062992125984)
('0x3e7', 0.0031496062992125984)


In [190]:
# view the result in pyviz
nt = Network()
nt.show_buttons()
nt.from_nx(G)
nt.show("nx.html")

In [28]:
# Betweenness centrality - shortest paths that pass through a particular node(broker node)
betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by betweenness centrality:")
for b in sorted_betweenness[:20]:
    print(b)



Top 20 nodes by betweenness centrality:
('wec.internal.cloudapp.net4', 0.0001742767514813524)
('MORDORDC.theshire.local340', 0.0)
('wec.internal.cloudapp.net340', 0.0)
('WORKSTATION5.theshire.local1648', 0.0)
('wec.internal.cloudapp.net1648', 0.0)
('-1648', 0.0)
('WORKSTATION6.theshire.local3120', 0.0)
('wec.internal.cloudapp.net3120', 0.0)
('WORKSTATION5.theshire.local3472', 0.0)
('wec.internal.cloudapp.net3472', 0.0)
('WORKSTATION6.theshire.local3548', 0.0)
('wec.internal.cloudapp.net3548', 0.0)
('MORDORDC.theshire.local3532', 0.0)
('wec.internal.cloudapp.net3532', 0.0)
('MORDORDC.theshire.local704', 0.0)
('wec.internal.cloudapp.net704', 0.0)
('MORDORDC.theshire.local0x0', 0.0)
('wec.internal.cloudapp.net0x0', 0.0)
('MORDORDC.theshire.local3616', 0.0)
('wec.internal.cloudapp.net3616', 0.0)


In [29]:
# Eigenvector centrality - useful for understanding which nodes can get information to many other nodes quickly. 
eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality
sorted_betweenness = sorted(eigenvector_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by betweenness centrality:")
for b in sorted_betweenness[:20]:
    print(b)

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')

In [106]:
uniqset = set()
for data in readfile:
    jsonobj = json.loads(data) 
    for keys in jsonobj.keys():
        tempp = keys.lower()
        matches = ["timestamp", "message"]
        if any(x in tempp for x in matches):
            uniqset.add(keys)

uniqset = list(uniqset)
uniqset

['Message', '@timestamp']

In [107]:
rjstimeline = []
for data in tqdm(readfile):
    jsonobj = json.loads(data)
    i = "@timestamp"
    j = "Message"
    temp = set()
    if i in jsonobj.keys() and j in jsonobj.keys():
        timeextract = str(jsonobj[i]).replace("T", " ").replace("Z", " ")[:-5]
        timeextract = datetime.strptime (timesample, "%Y-%m-%d %H:%M:%S")
        
        rjstimeline.append({"time": str(timeextract),"msg": str(jsonobj[j])[:100]})
len(rjstimeline)

100%|███████████████████████████████████████████████████████████████████████████| 6026/6026 [00:00<00:00, 22137.72it/s]


6026

In [108]:
rjstimeline

[{'time': '2020-08-07 14:32:25',
  'msg': 'Process accessed:\r\nRuleName: -\r\nUtcTime: 2020-08-07 14:32:22.692\r\nSourceProcessGUID: {9f85ce58-59dc-'},
 {'time': '2020-08-07 14:32:25',
  'msg': 'Process accessed:\r\nRuleName: -\r\nUtcTime: 2020-08-07 14:32:22.692\r\nSourceProcessGUID: {9f85ce58-59dc-'},
 {'time': '2020-08-07 14:32:25',
  'msg': 'Process accessed:\r\nRuleName: -\r\nUtcTime: 2020-08-07 14:32:22.692\r\nSourceProcessGUID: {9f85ce58-59dc-'},
 {'time': '2020-08-07 14:32:25',
  'msg': 'Registry object added or deleted:\r\nRuleName: -\r\nEventType: CreateKey\r\nUtcTime: 2020-08-07 14:32:22.8'},
 {'time': '2020-08-07 14:32:25',
  'msg': 'The Windows Filtering Platform has permitted a bind to a local port.\r\n\r\nApplication Information:\r\n\tP'},
 {'time': '2020-08-07 14:32:25',
  'msg': 'The Windows Filtering Platform has permitted a connection.\r\n\r\nApplication Information:\r\n\tProcess ID:'},
 {'time': '2020-08-07 14:32:25',
  'msg': 'Network connection detected:\r\nRuleN

In [109]:
with open(r"C:\Users\A101889\Desktop\Workspace\reactjs_visu\my-app\src\data.js", "w") as file:
    file.write("export const timeline_data = " + str(rjstimeline))

In [76]:
#filter by date time
rjstimeline = []
for data in tqdm(readfile):
    jsonobj = json.loads(data)
    i = "@timestamp"
    j = "Message"
    temp = set()
    if i in jsonobj.keys() and j in jsonobj.keys():
        timeextract = str(jsonobj[i]).replace("T", " ").replace("Z", " ")[:-5]
        timeextract = datetime.strptime (timesample, "%Y-%m-%d %H:%M:%S")
        
        rjstimeline.append({"time": str(timeextract),"msg": str(jsonobj[j])})
len(rjstimeline)

2020-08-07 14:32:25


In [100]:
filepath = r'C:\Users\A101889\Documents\GitHub\SecurityAnalytics\Playground\sample_dataset\mimikatz_logonpasswords_2020-08-07103224.json'
with open(filepath) as f:
    readfile = f.readlines()
    
edgeset = []
for item in tqdm(readfile):
    sampledata = json.loads(item)
    i = "SourceIp"
    j = "DestinationIp"
    
    if i in sampledata.keys() and j in sampledata.keys():
        edgepairs = (sampledata[i].split('\\')[-1], sampledata[j].split('\\')[-1])
        edgeset.append(edgepairs)
        
G = nx.Graph()
G.add_edges_from(edgeset)
        
G.nodes(data=True)
G.edges(data=True)

100%|███████████████| 6026/6026 [00:00<00:00, 39436.78it/s]


EdgeDataView([('172.18.39.5', '10.10.10.5', {}), ('172.18.39.5', '172.18.38.5', {}), ('0:0:0:0:0:0:0:1', '0:0:0:0:0:0:0:1', {}), ('172.18.38.5', '172.18.38.5', {}), ('fe80:0:0:0:3816:b2ee:1b9b:324b', 'fe80:0:0:0:3816:b2ee:1b9b:324b', {})])

In [101]:
data = {}
nodeList = []
linksList = []
for node in G.nodes(data=True):
    nodeList.append({"id": node[0]})
    
for link in G.edges(data=True):
    linksList.append({"source" : link[0], "target" : link[1]})
    
data = {
    "nodes" : nodeList,
    "links" : linksList
}

data

{'nodes': [{'id': '172.18.39.5'},
  {'id': '10.10.10.5'},
  {'id': '0:0:0:0:0:0:0:1'},
  {'id': '172.18.38.5'},
  {'id': 'fe80:0:0:0:3816:b2ee:1b9b:324b'}],
 'links': [{'source': '172.18.39.5', 'target': '10.10.10.5'},
  {'source': '172.18.39.5', 'target': '172.18.38.5'},
  {'source': '0:0:0:0:0:0:0:1', 'target': '0:0:0:0:0:0:0:1'},
  {'source': '172.18.38.5', 'target': '172.18.38.5'},
  {'source': 'fe80:0:0:0:3816:b2ee:1b9b:324b',
   'target': 'fe80:0:0:0:3816:b2ee:1b9b:324b'}]}